# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
#from agents.deals import ScrapedDeal, DealSelection
from jupyter_notebook_ai_clone.llm_udemy.llm_engineering.week8.agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

 20%|██        | 1/5 [01:34<06:17, 94.42s/it]/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week8/agents/deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|██████████| 5/5 [10:00<00:00, 120.15s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: All-Clad VIP Factory Seconds Sale: Up to 73% off + $9.95 shipping\nDetails: Shop factory seconds deals on All-Clad cookware, small appliances, knives, and more. Factory seconds deals may have cosmetic scratches, dents, or misprints that do not affect the performance of the cookware. You'll need to enter your email address to access this sale and all sales are final. Shipping adds $9.95 to all orders. Shop Now at Home & Cook\nFeatures: \nURL: https://www.dealnews.com/All-Clad-VIP-Factory-Seconds-Sale-Up-to-73-off-9-95-shipping/21721268.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung Q80D OLED TV Deals: Up to 48% off, up to a $300 drop from last week + free shipping
Details: Starting prices are as low as $750 for the 50" version (a $50 drop from last week). The 85" is $1,700 ($300 under last week's mention and almost half off at a discount of $1,600 off list).  Buy Now at Samsung
Features: 
URL: https://www.dealnews.com/Samsung-Q80-D-OLE

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The JLab JBuds Open Sport Air Conduction Wireless Earbuds are designed for active use, featuring IP55 sweatproof rating and secure earhooks that make them perfect for workouts and outdoor activities. These earbuds provide high-quality sound while allowing ambient noise, ensuring you stay aware of your surroundings. They are lightweight and comfortable, making them an excellent choice for anyone looking for reliable, workout-friendly audio gear.', price=50.0, url='https://www.dealnews.com/JLab-JBuds-Open-Sport-Air-Conduction-Wireless-Earbuds-for-50-free-shipping/21721263.html?iref=rss-c142')

In [16]:
#from agents.scanner_agent import ScannerAgent
from jupyter_notebook_ai_clone.llm_udemy.llm_engineering.week8.agents.scanner_agent import ScannerAgent

In [17]:
agent = ScannerAgent()
result = agent.scan()

/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week8/agents/deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()


In [18]:
result

DealSelection(deals=[Deal(product_description='The Samsung Q80D OLED TV offers stunning picture quality with its high contrast ratio and impressive colors. This model supports 4K resolution and is equipped with Quantum Dot technology, ensuring vivid imagery and sharp details. The sleek design fits well in any living room, and it includes Smart TV capabilities, allowing users to access popular streaming services effortlessly. The 50" version is priced at $750, offering a great entry point for high-end display technology.', price=750.0, url='https://www.dealnews.com/Samsung-Q80-D-OLED-TV-Deals-Up-to-48-off-up-to-a-300-drop-from-last-week-free-shipping/21721289.html?iref=rss-c142'), Deal(product_description='The Jackery Explorer 3000 Pro is a robust portable power station with a massive 3,024Wh capacity, making it ideal for camping or emergency backup. This unit comes with multiple outputs, including AC and USB ports, allowing you to power several devices simultaneously. Additionally, it 